# BNP-Paribas Classification Testing


In [1]:
# Import Necessary Libraries
import numpy as np
import os, csv, json

from matplotlib import *
from matplotlib import pyplot as plt

from sklearn import cross_validation
from sklearn.cross_validation import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# pretty charting
import seaborn as sns
sns.set_palette('muted')
sns.set_style('darkgrid')

%matplotlib inline

In [17]:
#### RUN AT BEGINNING AND TRY NOT TO RUN AGAIN - TAKES WAY TOO LONG ####
# load in test set to get labels
column = {}
targets = [] #np.array((), dtype=np.dtype('float64'))
with open('train.csv', 'rb') as file:
    reader = csv.reader(file)
    headers = reader.next()
    
    # get headers
    for h in headers:
        column[h] = []
    
    # append data
    for row in reader:
        # just create a matrix of all features
        targets.append(row)
        
        for h, v in zip(headers, row):
            # create a dict to access specific columns
            column[h].append(v)
            
# conver to a numpy matrix
targets = np.array(targets)

print targets.shape

(114321, 133)


In [12]:
#### RUN AT BEGINNING AND TRY NOT TO RUN AGAIN - TAKES WAY TOO LONG ####
# load in the feature data
column = {}
list_of_features = [] #np.array((), dtype=np.dtype('float64'))
with open('imputedData.csv', 'rb') as file:
    reader = csv.reader(file)
    headers = reader.next()
    
    # get headers
    for h in headers:
        column[h] = []
    
    # append data
    for row in reader:
        # just create a matrix of all features
        list_of_features.append(row)
        
        for h, v in zip(headers, row):
            # create a dict to access specific columns
            column[h].append(v)
            
# conver to a numpy matrix
list_of_features = np.array(list_of_features)

print list_of_features.shape
print len(column)

(228714, 131)
131


In [25]:
# create vector for targets - binary classification
targets = column['target'] 
features = list_of_features[0:len(targets), :]

targets = np.array(targets)
features = np.array(features)

print targets.shape
print features.shape

114321
(114321,)
(114321, 131)


In [4]:
np.random.seed(12345678)  # for reproducibility, set random seed

# define number of subjects per class
S = np.array((8, 16, 20, 32, 40, 64, 80, 100, 120, 200, 320,
              400, 800, 1000))

names = ["Nearest Neighbors", "Linear SVM", "Random Forest",
         "Linear Discriminant Analysis", "Quadratic Discriminant Analysis"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

In [27]:
accuracy=np.zeros((len(classifiers),2))
for idx, cla in enumerate(classifiers):
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, targets, test_size=0.4, random_state=0)
    clf = cla.fit(X_train, y_train)
    loo = LeaveOneOut(len(features))
    
    scores = cross_validation.cross_val_score(clf, features, targets, cv=loo)
    accuracy[idx,] = [scores.mean(), scores.std()]
    print("Accuracy of %s: %0.2f (+/- %0.2f)" % (names[idx], scores.mean(), scores.std() * 2))

## Discussion:
We need to downsample in order to run this analysis. There are way too many features and since the test size is so big, we need to downsample the features at least.

-> Plot correlation matrix and get the ones that aren't correlated and try it.